**Portfolio Optimization** -- Stock price prediction using RAG and LLM (*Harshita Patil*)

In [54]:
#here I imported necessary libraries
import yfinance as yf
import pandas as pd
from langchain_chroma import Chroma
import json,os,shutil
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
# from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from google.colab import userdata
userdata.get('HF_TOKEN')
userdata.get('OPENAI_API_KEY')

'""'

Gathered Stock Data Of Various Sector (*here 2 sector IT and PHARMA*)

In [ ]:
#used yahoo finance for this and get tickers of each company
it_tickers =['TCS.NS', 'INFY.NS', 'HCLTECH.NS']
pharma_tickers = ['CIPLA.NS', 'SUNPHARMA.NS', 'DRREDDY.NS']

In [ ]:
all_tickers = it_tickers + pharma_tickers

In [ ]:
#setting ending date and starting date to gather past 10 years of stock data
ending_date = pd.to_datetime('today').strftime('%Y-%m-%d')
starting_date = (pd.to_datetime('today')-pd.DateOffset(years = 10 )).strftime('%Y-%m-%d')

In [ ]:
#then I downloaded each data
try:
    data = yf.download(all_tickers,start=starting_date,end=ending_date)
    if data.empty:
        print("unable to get data")
        raise ValueError("Unable to download financial data from yfinance.")
    else:
        print("successfully downloaded data")
except Exception as e:
    print('an error occured: \n',e)


/tmp/ipython-input-3309111597.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_tickers,start=starting_date,end=ending_date)
[*********************100%***********************]  6 of 6 completed

successfully downloaded data


In [ ]:
# Here I made he data suitable for accurate result using Adj_close so that if any event occurs and if there is sudden changes in the stock value it will get equlize
if not data.empty:
    Adj_close_df =pd.DataFrame()

    # financial_statements = pd.DataFrame()
    # balance_sheet
    for ticker in all_tickers:
        try:
            data = yf.download(ticker,start=starting_date,end=ending_date)
            Adj_close_df[ticker] = data['Close'][ticker]
            if data.empty:
                print("unable to get data")
            else:
                print(f"successfully downloaded data for {ticker}")
        except Exception as e:
            print('an error occured: \n',e)

/tmp/ipython-input-3930525239.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker,start=starting_date,end=ending_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3930525239.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker,start=starting_date,end=ending_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3930525239.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker,start=starting_date,end=ending_date)


successfully downloaded data for TCS.NS
successfully downloaded data for INFY.NS


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3930525239.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker,start=starting_date,end=ending_date)
[*********************100%***********************]  1 of 1 completed

successfully downloaded data for HCLTECH.NS



/tmp/ipython-input-3930525239.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker,start=starting_date,end=ending_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3930525239.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker,start=starting_date,end=ending_date)
[*********************100%***********************]  1 of 1 completed


successfully downloaded data for CIPLA.NS
successfully downloaded data for SUNPHARMA.NS
successfully downloaded data for DRREDDY.NS


In [ ]:
print(Adj_close_df)

                 TCS.NS      INFY.NS   HCLTECH.NS     CIPLA.NS  SUNPHARMA.NS  \
Date                                                                           
2015-09-07  1085.512939   414.411774   354.804443   611.157227    771.010925   
2015-09-08  1085.747925   414.431366   360.776611   611.580627    778.476807   
2015-09-09  1089.786743   426.134827   363.415924   617.649475    773.361267   
2015-09-10  1087.927734   422.905579   352.897247   615.062012    773.960327   
2015-09-11  1090.748535   426.937134   354.553955   614.544556    779.813293   
...                 ...          ...          ...          ...           ...   
2025-09-01  3112.600098  1499.400024  1468.400024  1587.199951   1563.300049   
2025-09-02  3111.500000  1498.699951  1465.699951  1569.199951   1565.199951   
2025-09-03  3097.500000  1479.099976  1466.099976  1578.400024   1578.699951   
2025-09-04  3095.699951  1463.000000  1443.300049  1577.800049   1582.300049   
2025-09-05  3048.300049  1444.599976  14

In [ ]:
# I looped on each ticker object to gather information about company, recent news,recommendation using ticker object.
vector_db_data =[]
for ticker in all_tickers:
    try:
        #ticker obj for curr stock
        ticker_object = yf.Ticker(ticker)
        ticker_historical_prices = Adj_close_df[ticker].to_json()

        # current info & recommendation of ticker
        info = ticker_object.info
        news = ticker_object.news
        recommendations = ticker_object.recommendations
        stock_data = {
            "ticker": ticker,
            "sector": info.get('sector', 'N/A'),
            "long_name": info.get('longName', 'N/A'),

            # Adj Close
            "historical_adj_close": ticker_historical_prices,

            #  fundamental data
            "fundamental_data": {
                "P/E Ratio": info.get('forwardPE'),
                "Market Cap": info.get('marketCap'),
                "Dividend Yield": info.get('dividendYield'),
                "Beta": info.get('beta'),
                "52 Week High": info.get('fiftyTwoWeekHigh')
            },

            "news": news,
            "analyst_recommendations": recommendations.to_json() if recommendations is not None else None
        }
        print(stock_data)
        vector_db_data.append(stock_data)

    except Exception as e:
        print(f"\nAn error occurred while getting data for {ticker}: {e}")

{'ticker': 'TCS.NS', 'sector': 'Technology', 'long_name': 'Tata Consultancy Services Limited', 'historical_adj_close': '{"1441584000000":1085.5129394531,"1441670400000":1085.7479248047,"1441756800000":1089.7867431641,"1441843200000":1087.927734375,"1441929600000":1090.7485351562,"1442188800000":1090.5989990234,"1442275200000":1084.4440917969,"1442361600000":1090.0434570312,"1442534400000":1090.4281005859,"1442793600000":1090.3424072266,"1442880000000":1079.9776611328,"1442966400000":1082.4780273438,"1443052800000":1102.8863525391,"1443398400000":1108.5708007812,"1443484800000":1106.2419433594,"1443571200000":1106.1567382812,"1443657600000":1130.3266601562,"1444003200000":1158.9849853516,"1444089600000":1154.0053710938,"1444176000000":1134.3227539062,"1444262400000":1125.7103271484,"1444348800000":1123.9371337891,"1444608000000":1108.1014404297,"1444694400000":1110.6444091797,"1444780800000":1061.5561523438,"1444867200000":1054.7391357422,"1444953600000":1056.5561523438,"1445212800000":

In [ ]:
#created a separate json file
with open('financial_data_full.json', 'w') as f:
    json.dump(vector_db_data, f, indent=4)

In [ ]:
with open('financial_data_full.json', 'r') as f:
            json_data = json.load(f)

In [ ]:
df = pd.DataFrame(json_data)

In [ ]:
print(df.shape)

(6, 7)


In [ ]:
# !pip install langchain_chroma
!pip install langchain-openai

In [ ]:
# creating document for loading it into vector db
documents = []
for item in json_data:
    content = (
        f"Ticker: {item['ticker']}\n"
        f"Long Name: {item['long_name']}\n"
        f"Sector: {item['sector']}\n"
        f"Fundamental Data: {json.dumps(item['fundamental_data'])}\n"
        f"News: {json.dumps(item['news'])}\n"
        f"Analyst Recommendations: {item['analyst_recommendations']}"
    )
    doc = Document(
        page_content=content,
        metadata={
            "ticker": item['ticker'],
            "source": "yfinance_data",
            "long_name": item['long_name'],
            "historical_adj_close": item['historical_adj_close'] # Store the full data here for retrieval
        }
    )
    documents.append(doc)
print(f"Created {len(documents)} Document objects.")

Created 6 Document objects.


In [ ]:
!pip install langchain-chroma

In [ ]:
# !pip install -U langchain-huggingface

here at first i tried to use huggingfaceembeddings,used quant tokenization for llm but huggingfaceembeddings didn't support large token inputs

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# import os
# from langchain.llms import HuggingFacePipeline
# from torch import cuda, bfloat16,float16
# import transformers
# from transformers import AutoTokenizer , BitsAndBytesConfig
# from langchain.chains import RetrievalQA
# from langchain.prompts import PromptTemplate
# from langchain_chroma import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.llms import HuggingFacePipeline
# import pandas as pd
# from langchain.chains import RetrievalQA
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.llms import HuggingFacePipeline

# hg_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# persist_directory = '/chromadbforembedding/'
# import os , shutil
# # Delete the existing Chroma DB directory if it exists
# if os.path.exists(persist_directory):
#     shutil.rmtree(persist_directory)
#     print(f"Deleted existing Chroma DB at {persist_directory}")


# # Create and initialize the Chroma DB
# langchain_vector_db = Chroma.from_documents(
#     documents=documents,
#     collection_name="portfolio-optimization",
#     embedding=hg_embeddings,
#     persist_directory=persist_directory
# )


In [ ]:
# !pip install -U bitsandbytes accelerate transformers

In [ ]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=bfloat16
# )
# model = transformers.AutoModelForCausalLM.from_pretrained(
#     model_id,
#     # trust_remote_code=True,
#     quantization_config=bnb_config,
#     device_map="cpu",
#     trust_remote_code=True
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# print(tokenizer)


In [ ]:
# from torch import cuda, bfloat16,float16
# import transformers
# from transformers import AutoTokenizer , BitsAndBytesConfig
# from langchain.llms import HuggingFacePipeline



In [ ]:
# model_id = "gpt-5"
# device = 'cpu'


In [ ]:
# !pip install -U bitsandbytes accelerate transformers



In [ ]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=bfloat16
# )
# model = transformers.AutoModelForCausalLM.from_pretrained(
#     model_id,
#     # trust_remote_code=True,
#     quantization_config=bnb_config,
#     device_map="cpu",
#     trust_remote_code=True
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# print(tokenizer)
# query_pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     dtype=float16,
#     device_map="cpu",
#     # max__new_tokens=500
# )
# llm = HuggingFacePipeline(pipeline=query_pipeline)


In [ ]:
# !pip install bitsandbytes

In [ ]:
# !pip install -U bitsandbytes


I used openai embeddings to create vector database--> converting text content to vector embeddings

In [ ]:
openai_key = os.getenv('OPENAI_API_KEY')
# os.environ["OPENAI_API_KEY"] = ""
openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# hg_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# import os

Initializing chroma db

In [ ]:
persist_directory = '/chromadbforembedding2/'

if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
    print(f"Deleted existing Chroma DB at {persist_directory}")


# here i'm initializing chroma db
langchain_vector_db = Chroma.from_documents(
    documents=documents,
    collection_name="portfolio-optimization",
    embedding=openai_embeddings,
    persist_directory=persist_directory
)
print("Successfully created and persisted Chroma DB.")

Successfully created and persisted Chroma DB.


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


Defining prompt template for llm

In [ ]:
#using chat model of openai
llm = ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
import pandas as pd


In [ ]:
template = """You are a financial analyst. Use the provided historical data of past 10 years, current news and fundamental data to predict the return value after/on given time period. if the user's portfolio has multiple values you take average of the return value and give the final return value.
You have access to historical data for various stocks. The data is in JSON format.
If you cannot find the relevant information in the provided context, state that you do not have enough information.

Context:
{metadata}

Question:
{query}

Answer:(should be short only the return value and how it is predicted reason in short 1 sentence)
EXAMPLE: return value:, reason:
"""
PROMPT =  PromptTemplate(
    input_variables=["metadata", "query"],template=template
)
print(PROMPT)



input_variables=['metadata', 'query'] input_types={} partial_variables={} template="You are a financial analyst. Use the provided historical data of past 10 years, current news and fundamental data to predict the return value after/on given time period. if the user's portfolio has multiple values you take average of the return value and give the final return value.\nYou have access to historical data for various stocks. The data is in JSON format.\nIf you cannot find the relevant information in the provided context, state that you do not have enough information.\n\nContext:\n{metadata}\n\nQuestion:\n{query}\n\nAnswer:(should be short only the return value and how it is predicted reason in short 1 sentence)\nEXAMPLE: return value:, reason:\n"


In [ ]:
#RETRIEVER SETUP THAT IS QUERING THE VECTOR DB
retriever = langchain_vector_db.as_retriever(search_kwargs={"k":3})


In [ ]:
#Funtion to remove the duplicate documents retrieved from the rag
def remove_duplicates(document):
  seen = set()
  unique_docs = []
  for doc in document:
    identifier = (doc.page_content, tuple(sorted(doc.metadata.items())))
    if identifier not in seen:
            seen.add(identifier)
            unique_docs.append(doc)
  return unique_docs

In [ ]:
#skipped using RetrievalQA as it was too resource-intensive leading to kernel crashes and high RAM usage
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     # chain_type="stuff",
#     retriever=retriever,
#     # return_source_documents=True,
#     chain_type_kwargs={"prompt": PROMPT}
# )

#switched to llmchain which is lightweight, to provide answer based on rag input and prompt template
qa_chain = LLMChain(llm=llm, prompt=PROMPT)

/tmp/ipython-input-4068073646.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=PROMPT)


Function to do rag search and formating the result to pass to llm

In [ ]:
def get_answer(query):
    #------RAG SEARCH based on user query----
    raw_docs = retriever.get_relevant_documents(query)

    # print(raw_docs)
    unique_docs = remove_duplicates(raw_docs)

    context_list = []
    for doc in unique_docs:
        ticker = doc.metadata.get('ticker', 'N/A')
        long_name = doc.metadata.get('long_name', 'N/A')
        historical_adj_close = doc.metadata.get('historical_adj_close', '{}')

        # ----I Combine different parts,to create readable format for the LLM---
        context_string = f"Ticker: {ticker}\nLong Name: {long_name}\n"
        context_string += f"Document Content: {doc.page_content}\n"
        context_string += f"Historical Adjusted Close Prices: {historical_adj_close}\n"

        context_list.append(context_string)


    formatted_context = "\n---\n".join(context_list)
    print(type(formatted_context))

    # answer = qa_chain({"query": query, "context": formatted_context})
    #----GENERATING THE ANSWER USING LLM----
    answer = qa_chain.invoke({
    "query": query,
    "metadata": formatted_context
    })
    # answer = qa_chain({"query":query,"context":context_parts})
    print(PROMPT)
    print(answer)
    return answer['text']


In [ ]:
query = "IF I invest 30% in TCS , my INR 1000 revenue now what will be the return after 1 year"
response = get_answer(query)
print(response)

/tmp/ipython-input-1594171026.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw_docs = retriever.get_relevant_documents(query)


<class 'str'>
input_variables=['metadata', 'query'] input_types={} partial_variables={} template="You are a financial analyst. Use the provided historical data of past 10 years, current news and fundamental data to predict the return value after/on given time period. if the user's portfolio has multiple values you take average of the return value and give the final return value.\nYou have access to historical data for various stocks. The data is in JSON format.\nIf you cannot find the relevant information in the provided context, state that you do not have enough information.\n\nContext:\n{metadata}\n\nQuestion:\n{query}\n\nAnswer:(should be short only the return value and how it is predicted reason in short 1 sentence)\nEXAMPLE: return value:, reason:\n"
{'query': 'IF I invest 30% in TCS , my INR 1000 revenue now what will be the return after 1 year', 'metadata': 'Ticker: TCS.NS\nLong Name: Tata Consultancy Services Limited\nDocument Content: Ticker: TCS.NS\nLong Name: Tata Consultanc

In [ ]:
query = "currently I have 1000 INR, I want to invest 30% in CIPLA, 70% in TCS, what is the value of return after 3 years "
response = get_answer(query)
print(response)

<class 'str'>
input_variables=['metadata', 'query'] input_types={} partial_variables={} template="You are a financial analyst. Use the provided historical data of past 10 years, current news and fundamental data to predict the return value after/on given time period. if the user's portfolio has multiple values you take average of the return value and give the final return value.\nYou have access to historical data for various stocks. The data is in JSON format.\nIf you cannot find the relevant information in the provided context, state that you do not have enough information.\n\nContext:\n{metadata}\n\nQuestion:\n{query}\n\nAnswer:(should be short only the return value and how it is predicted reason in short 1 sentence)\nEXAMPLE: return value:, reason:\n"
{'query': 'currently I have 1000 INR, I want to invest 30% in CIPLA, 70% in TCS, what is the value of return after 3 years ', 'metadata': 'Ticker: TCS.NS\nLong Name: Tata Consultancy Services Limited\nDocument Content: Ticker: TCS.NS\

In [ ]:
query = "currently I have 1000 INR, I want to invest 30% in CIPLA what is the value of return after 3 years "
response = get_answer(query)
print(response)

<class 'str'>
input_variables=['metadata', 'query'] input_types={} partial_variables={} template="You are a financial analyst. Use the provided historical data of past 10 years, current news and fundamental data to predict the return value after/on given time period. if the user's portfolio has multiple values you take average of the return value and give the final return value.\nYou have access to historical data for various stocks. The data is in JSON format.\nIf you cannot find the relevant information in the provided context, state that you do not have enough information.\n\nContext:\n{metadata}\n\nQuestion:\n{query}\n\nAnswer:(should be short only the return value and how it is predicted reason in short 1 sentence)\nEXAMPLE: return value:, reason:\n"
{'query': 'currently I have 1000 INR, I want to invest 30% in CIPLA what is the value of return after 3 years ', 'metadata': 'Ticker: CIPLA.NS\nLong Name: Cipla Limited\nDocument Content: Ticker: CIPLA.NS\nLong Name: Cipla Limited\nSe

In [ ]:
query = "currently I have 1000 INR, I want to invest 50% in HCLTECH and 50% in TCS  what is the value of return after 3 years "
response = get_answer(query)
print(response)

<class 'str'>
input_variables=['metadata', 'query'] input_types={} partial_variables={} template="You are a financial analyst. Use the provided historical data of past 10 years, current news and fundamental data to predict the return value after/on given time period. if the user's portfolio has multiple values you take average of the return value and give the final return value.\nYou have access to historical data for various stocks. The data is in JSON format.\nIf you cannot find the relevant information in the provided context, state that you do not have enough information.\n\nContext:\n{metadata}\n\nQuestion:\n{query}\n\nAnswer:(should be short only the return value and how it is predicted reason in short 1 sentence)\nEXAMPLE: return value:, reason:\n"
{'query': 'currently I have 1000 INR, I want to invest 50% in HCLTECH and 50% in TCS  what is the value of return after 3 years ', 'metadata': 'Ticker: TCS.NS\nLong Name: Tata Consultancy Services Limited\nDocument Content: Ticker: TC

In [ ]:
query = "currently I have 1000 INR, I want to invest 50% in SUNPHARMA and 50% in TCS  what is the value of return after 3 years "
response = get_answer(query)
print(response)

<class 'str'>
input_variables=['metadata', 'query'] input_types={} partial_variables={} template="You are a financial analyst. Use the provided historical data of past 10 years, current news and fundamental data to predict the return value after/on given time period. if the user's portfolio has multiple values you take average of the return value and give the final return value.\nYou have access to historical data for various stocks. The data is in JSON format.\nIf you cannot find the relevant information in the provided context, state that you do not have enough information.\n\nContext:\n{metadata}\n\nQuestion:\n{query}\n\nAnswer:(should be short only the return value and how it is predicted reason in short 1 sentence)\nEXAMPLE: return value:, reason:\n"
{'query': 'currently I have 1000 INR, I want to invest 50% in SUNPHARMA and 50% in TCS  what is the value of return after 3 years ', 'metadata': 'Ticker: TCS.NS\nLong Name: Tata Consultancy Services Limited\nDocument Content: Ticker: 

In [ ]:
query = "currently I have 3000 INR, I want to invest 20% in SUNPHARMA and 50% in TCS  what is the value of return after 3 years "
response = get_answer(query)
print(response)

<class 'str'>
input_variables=['metadata', 'query'] input_types={} partial_variables={} template="You are a financial analyst. Use the provided historical data of past 10 years, current news and fundamental data to predict the return value after/on given time period. if the user's portfolio has multiple values you take average of the return value and give the final return value.\nYou have access to historical data for various stocks. The data is in JSON format.\nIf you cannot find the relevant information in the provided context, state that you do not have enough information.\n\nContext:\n{metadata}\n\nQuestion:\n{query}\n\nAnswer:(should be short only the return value and how it is predicted reason in short 1 sentence)\nEXAMPLE: return value:, reason:\n"
{'query': 'currently I have 3000 INR, I want to invest 20% in SUNPHARMA and 50% in TCS  what is the value of return after 3 years ', 'metadata': 'Ticker: TCS.NS\nLong Name: Tata Consultancy Services Limited\nDocument Content: Ticker: 